In [46]:
import pandas as pd
import numpy as np
from repo_funciones import * #repositorio propio de funciones
import re #para expresiones regulares

In [47]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
data = pd.read_csv('../Sharknado/attacks_messy.csv', encoding='utf-8',encoding_errors="ignore" )

In [48]:
"""Constraints
- Hay filas que agrupan a varias personas en vez de ser valores unicos. A efectos de este ejercicio, se tomaran como valor unico.

"""

'Constraints\n- Hay filas que agrupan a varias personas en vez de ser valores unicos. A efectos de este ejercicio, se tomaran como valor unico.\n\n'

Exploración Inicial

In [49]:
dim_original = data.shape #Registramos dimensiones iniciales del archivo para luego ir comparando
dim_original

(25723, 24)

In [50]:
#data.head() Usamos estas funciones para ir haciendonos una idea de los datos y las columnas
#data.tail() Con este nos damos cuenta de que al final del archivo hay filas enteras con valores nulos

In [51]:
#data.info() #Estimación de que columnas necesitan mas trabajo. Por gusto mío prefiero verlo a la inversa -> que columnas tienen mas valores nulos.

In [52]:
nancols(data) #Conteo de valores nulos por col, ascending False.

Unnamed: 22               25722
Unnamed: 23               25721
Time                      22775
Species                   22259
Age                       22252
Sex                       19986
Activity                  19965
Location                  19961
Fatal (Y/N)               19960
Area                      19876
Name                      19631
Country                   19471
Injury                    19449
Investigator or Source    19438
Type                      19425
Year                      19423
href formula              19422
Date                      19421
pdf                       19421
href                      19421
Case Number.1             19421
Case Number.2             19421
original order            19414
Case Number               17021
dtype: int64

In [53]:
nancolsper(data) #conteo de valores nulos como % del total de la columna. Nos da una mejor estimación 

Unnamed: 22               100.00
Unnamed: 23                99.99
Time                       88.54
Species                    86.53
Age                        86.51
Sex                        77.70
Activity                   77.62
Location                   77.60
Fatal (Y/N)                77.60
Area                       77.27
Name                       76.32
Country                    75.69
Injury                     75.61
Investigator or Source     75.57
Type                       75.52
Year                       75.51
href formula               75.50
Date                       75.50
pdf                        75.50
href                       75.50
Case Number.1              75.50
Case Number.2              75.50
original order             75.47
Case Number                66.17
dtype: float64

Limpieza duplicados y columnas vacias

In [54]:
data= data.drop_duplicates()

In [55]:
data.duplicated().unique()

array([False])

In [56]:
data=data.drop(nanrows(data)[nanrows(data)> 19]) #Si en una fila 90% de los valores son nulos, se descartan

In [57]:
data = data.reset_index(drop=True)

In [58]:
data = data.drop(nanrows(data)[nanrows(data) >= 15]) #Si en una fila 60% de los valores son nulos, se descartan.

In [59]:
dim_original, data.shape

((25723, 24), (6306, 24))

In [60]:
data = data.reset_index(drop=True)

In [61]:
nancolsper(data) #Unamed 22 y 23 estan vacías y no aportan nada. Se descartan.

Unnamed: 22               99.98
Unnamed: 23               99.97
Time                      53.30
Species                   45.15
Age                       45.04
Sex                        9.12
Activity                   8.79
Location                   8.72
Fatal (Y/N)                8.69
Area                       7.37
Name                       3.49
Country                    0.95
Injury                     0.60
Investigator or Source     0.43
Type                       0.22
Year                       0.19
href formula               0.17
Date                       0.16
pdf                        0.16
href                       0.16
Case Number.1              0.16
Case Number.2              0.16
original order             0.05
Case Number                0.03
dtype: float64

In [62]:
data.drop(["Unnamed: 22", "Unnamed: 23"], axis=1, inplace=True)  #Reminder, axis 0 son filas y axis columnas

In [63]:
nancolsper(data) #Time,species y age, la proporción de valores nulos es demasiado representativa para poder rellenar con media/moda ya que introduciría un sesgo muy importante.

Time                      53.30
Species                   45.15
Age                       45.04
Sex                        9.12
Activity                   8.79
Location                   8.72
Fatal (Y/N)                8.69
Area                       7.37
Name                       3.49
Country                    0.95
Injury                     0.60
Investigator or Source     0.43
Type                       0.22
Year                       0.19
href formula               0.17
Date                       0.16
Case Number.2              0.16
pdf                        0.16
href                       0.16
Case Number.1              0.16
original order             0.05
Case Number                0.03
dtype: float64

In [64]:
data.columns = data.columns.str.strip().str.capitalize().str.title().str.replace(':','').str.replace(' ','_').str.replace('.','_')

Limpieza Columna "Time"

In [65]:
#data["Time"].unique() #Vemos hay muchas inconsistencias en los valores, hay que homogenizar

In [66]:
#unique_cols(data) #364 valores unicos 
#get_unique_dtypes(data, "Time") #con strings y con floats. Se intuye floats seran los valores nulos.

In [67]:
data["Time"]=data["Time"].fillna("unknown")

In [68]:
data["Time"] = data["Time"].str.replace('h',':')

In [69]:
#convención de tiempo
#Morning: 6am a 12
#afternoon: 12 a 7
#night: 8 a 5am
#Asumimos que todos ingresaron la hora en formato 24h
data.loc[data["Time"]=="14:00  -15:00", "Time"]="14:30"
data.loc[data["Time"]=="Late afternoon", "Time"]="18:00"
data.loc[data["Time"]=="Morning", "Time"]="09:00"
data.loc[data["Time"]=="19:00, Dusk", "Time"]="19:00"
data.loc[data["Time"]=="Nig:t", "Time"]="21:00"
data.loc[data["Time"]=="Afternoon", "Time"]="17:00"
data.loc[data["Time"]=="Midday", "Time"]="13:00"
data.loc[data["Time"]=="S:ortly before 12:00", "Time"]="11:30"
data.loc[data["Time"]=="After noon", "Time"]="13:00"
data.loc[data["Time"]=="14:30 / 15:30", "Time"]="15:00"
data.loc[data["Time"]=="Midnig:t", "Time"]="24:00"
data.loc[data["Time"]=="09:30 / 10:00", "Time"]="09:40"
data.loc[data["Time"]=="10:45-11:15", "Time"]="11:00"
data.loc[data["Time"]=="Evening", "Time"]="20:00"
data.loc[data["Time"]=="Sometime between 06:00 & 08:oo", "Time"]="07:00"
data.loc[data["Time"]=="Early afternoon", "Time"]="14:00"
data.loc[data["Time"]=="18:15-18:30", "Time"]="18:20"
data.loc[data["Time"]=="09:00 - 09:30", "Time"]="09:15"
data.loc[data["Time"]=="0830", "Time"]="08:30"
data.loc[data["Time"]=="Just before noon", "Time"]="11:30"
data.loc[data["Time"]=="1600", "Time"]="16:00"
data.loc[data["Time"]=="Early morning", "Time"]="09:00"
data.loc[data["Time"]=="Dawn", "Time"]="07:00"
data.loc[data["Time"]=="AM", "Time"]="07:00"
data.loc[data["Time"]=="A.M", "Time"]="07:00"
data.loc[data["Time"]=="Dusk", "Time"]="20:00"
data.loc[data["Time"]=="Lunc:time", "Time"]="13:00"
data.loc[data["Time"]=="15j45", "Time"]="15:45"
data.loc[data["Time"]=="0500", "Time"]="05:00"
data.loc[data["Time"]=="Before 07:00", "Time"]="06:00"
data.loc[data["Time"]=="10:00 -- 11:00", "Time"]="10:30"
data.loc[data["Time"]=='"Just before 11:00"', "Time"]="10:30"
data.loc[data["Time"]=="Sunset", "Time"]="19:30"
data.loc[data["Time"]=="Just before sundown", "Time"]="19:30"
data.loc[data["Time"]=="Between 05:00 and 08:00", "Time"]="06:30"
data.loc[data["Time"]=="17:00 or 17:40", "Time"]="17:20"
data.loc[data["Time"]==">08:00", "Time"]="08:20"
data.loc[data["Time"]=="--", "Time"]="unknown"
data.loc[data["Time"]=="Just after 12:00", "Time"]="12:30"
data.loc[data["Time"]=="Early Morning", "Time"]="08:30"
data.loc[data["Time"]=="S:ortly after midnig:t", "Time"]="24:30"
data.loc[data["Time"]=="09:00 -10:00", "Time"]="09:30"
data.loc[data["Time"]=="20:45 (Sunset)", "Time"]="20:45"
data.loc[data["Time"]=="Late morning", "Time"]="11:00"
data.loc[data["Time"]=="P.M.", "Time"]="14:00"
data.loc[data["Time"]=="S:ortly before 13:00", "Time"]="12:00"
data.loc[data["Time"]=="8:04 pm", "Time"]="08:00"
data.loc[data["Time"]=="Possibly same incident as 2000.08.21", "Time"]="unknown"
data.loc[data["Time"]=="After Dusk", "Time"]="09:00"
data.loc[data["Time"]=="Noon", "Time"]="12:00"
data.loc[data["Time"]=="2 :ours after Opperman", "Time"]="unknown"
data.loc[data["Time"]=="Mid afternoon", "Time"]="14:00"
data.loc[data["Time"]=="Mid morning", "Time"]="11:00"
data.loc[data["Time"]=="11:00 / 11:30", "Time"]="11:00"
data.loc[data["Time"]=='"Nig:t"', "Time"]="21:00"
data.loc[data["Time"]=='18:30?', "Time"]="18:30"
data.loc[data["Time"]=='30 minutes after 1992.07.08.a', "Time"]="unknown"
data.loc[data["Time"]==">06:45", "Time"]="06:45"
data.loc[data["Time"]=="Between 06:00 & 07:20", "Time"]="06:45"
data.loc[data["Time"]=="<07:30", "Time"]="06:45"
data.loc[data["Time"]=="17:00 Sunset", "Time"]="17:00"
data.loc[data["Time"]=="Nig:tfal", "Time"]="20:00"
data.loc[data["Time"]=="X", "Time"]="unknown"
data.loc[data["Time"]=="18:30 (Sunset)", "Time"]="18:30"
data.loc[data["Time"]=="06j00", "Time"]="06:00"
data.loc[data["Time"]=="Prior to 10:37", "Time"]="10:00"
data.loc[data["Time"]=="Daybreak", "Time"]="20:00"
data.loc[data["Time"]==">12:00", "Time"]="12:00"
data.loc[data["Time"]=="Mid-morning", "Time"]="11:00"
data.loc[data["Time"]=="16:30 or 18:00", "Time"]="18:00"
data.loc[data["Time"]=="Just before dawn", "Time"]="19:00"
data.loc[data["Time"]=="Daytime", "Time"]="10:00"
data.loc[data["Time"]=="Dark", "Time"]="23:00"
data.loc[data["Time"]=="10:00 / 11:00", "Time"]="10:00"
data.loc[data["Time"]=='"After lunc:"', "Time"]="16:00"
data.loc[data["Time"]=='15:00 or 15:45', "Time"]="15:00"
data.loc[data["Time"]=='>17:00', "Time"]="17:00"
data.loc[data["Time"]=='19:00 / 20:00', "Time"]="19:00"
data.loc[data["Time"]=="12:45 / 13:45", "Time"]="13:00"
data.loc[data["Time"]=="14:00 - 15:00", "Time"]="14:00"
data.loc[data["Time"]=="nig:t", "Time"]="20:00"
data.loc[data["Time"]=="03:45 - 04:00", "Time"]="03:00"
data.loc[data["Time"]=="Late nig:t", "Time"]="23:00"
data.loc[data["Time"]=="10:30 or 13:30", "Time"]="12:00"
data.loc[data["Time"]=="15:00j", "Time"]="15:00"
data.loc[data["Time"]=="Midday.", "Time"]="12:00"
data.loc[data["Time"]=='"After dark"', "Time"]="23:00"
data.loc[data["Time"]=='10:00 or 14:00"', "Time"]="23:00"
data.loc[data["Time"]=="2 :rs before sunset", "Time"]="16:00"
data.loc[data["Time"]=="18:15 to 21:30", "Time"]="16:00"
data.loc[data["Time"]=="1500", "Time"]="15:00"
data.loc[data["Time"]=="A.M.", "Time"]="09:00"
data.loc[data["Time"]=="Morning ", "Time"]="09:00"
data.loc[data["Time"]=="07:00 - 08:00", "Time"]="07:00"
data.loc[data["Time"]=="  ", "Time"]="unknown"
data.loc[data["Time"]=='"Evening"', "Time"]="unknown"
data.loc[data["Time"]=="11:30 ", "Time"]="11:30"
data.loc[data["Time"]==' ', "Time"]="unknown"
data.loc[data["Time"]=="Nig:tfall", "Time"]="21:00"
data.loc[data["Time"]=="09:30 / 15:30", "Time"]="12:00"
data.loc[data["Time"]=="12:00 to 14:00", "Time"]="13:00"
data.loc[data["Time"]=="10:00 or 14:00", "Time"]="13:00"
data.loc[data["Time"]=='"s:ortly before dusk"', "Time"]="20:00"
data.loc[data["Time"]==">17:30", "Time"]="18:00"
data.loc[data["Time"]==">14:30", "Time"]="15:00"
data.loc[data["Time"]=="Between 11:00 & 12:00", "Time"]="11:00"
data.loc[data["Time"]=="After 04:00", "Time"]="05:00"
data.loc[data["Time"]=="11:01 -time of s:ip sinking", "Time"]="11:00"
data.loc[data["Time"]=="S:ip aban-doned at 03:10", "Time"]="03:00"
data.loc[data["Time"]=="After dusk", "Time"]="22:00"
data.loc[data["Time"]=="FATAL  (Wire netting installed at local beac:es after t:is incident.)", "Time"]="unknown"
data.loc[data["Time"]=="After midnig:t", "Time"]="01:00"
data.loc[data["Time"]=="Late afternon", "Time"]="18:00"
data.loc[data["Time"]=='"Early evening"', "Time"]="20:00"
data.loc[data["Time"]=="Late Afternoon", "Time"]="18:00"
data.loc[data["Time"]=="   ", "Time"]="unknown"
data.loc[data["Time"]=="Before daybreak", "Time"]="19:00"
data.loc[data["Time"]=="dusk", "Time"]="20:00"
data.loc[data["Time"]=="Before 10:30", "Time"]="20:00"
data.loc[data["Time"]=="06:00 -- 07:00", "Time"]="06:00"
data.loc[data["Time"]=="17:00-18:00", "Time"]="17:00"
data.loc[data["Time"]=="19:00-20:00", "Time"]="19:00"
data.loc[data["Time"]=="1300", "Time"]="13:00"
data.loc[data["Time"]=="11:115", "Time"]="11:00"
data.loc[data["Time"]=="13:345", "Time"]="11:00"
data.loc[data["Time"]=="08:00 / 09:30", "Time"]="08:00"
data.loc[data["Time"]==" 14:00", "Time"]="14:00"

In [70]:
#note to self, hubieses empezado quitando valores especiales/espaciosR.L y te hubieses ahorrado la mitad de este chorizo :,)
#data["Time"].unique()
get_unique_dtypes(data, "Time")

array([<class 'str'>], dtype=object)

In [71]:
data["Time_groups"] = data['Time'].apply(time_of_day)

In [72]:
col_unique_counts(data, "Time_groups")

,Time_groups,Count
0,No information,3378
1,Afternoon,1702
2,Morning,1124
3,Night,102


Limpieza especies 

In [73]:
data["Species"]=data["Species"].fillna("unknown")

Limpieza "Age"

In [74]:
#quitamos espacios y caracteres especiales
data["Age"] = data["Age"].str.replace(r'[^\w\s]|[\&\(\)\.\"&gt;&lt;\?\']', '', regex=True)


In [75]:
data["Age"] = data["Age"].str.rstrip()
data["Age"] = data["Age"].str.lstrip()


In [76]:
data["Age"]=data["Age"].fillna("0")

In [77]:
data.loc[data["Age"]=="Teen", "Age"]="17"
data.loc[data["Age"]=="40s", "Age"]="40"
data.loc[data["Age"]=="60s", "Age"]="60"
data.loc[data["Age"]=="18 monhs", "Age"]="1"
data.loc[data["Age"]=="een", "Age"]="17"
data.loc[data["Age"]=="18 or 20", "Age"]="20"
data.loc[data["Age"]=="30s", "Age"]="30" #mental note: se podria haber hecho algo para quitar toda las "s"
data.loc[data["Age"]=="50s", "Age"]="50"
data.loc[data["Age"]=="28  26", "Age"]="27"
data.loc[data["Age"]=="12 or 13", "Age"]="13"
data.loc[data["Age"]=="46  34", "Age"]="40"
data.loc[data["Age"]=="28 23  30", "Age"]="25"
data.loc[data["Age"]=="36  26", "Age"]="30"
data.loc[data["Age"]=="8 or 10", "Age"]="9"
data.loc[data["Age"]=="30 or 36", "Age"]="32"
data.loc[data["Age"]=="33 or 37", "Age"]="35"
data.loc[data["Age"]=="mid30s", "Age"]="35"
data.loc[data["Age"]=="23  20", "Age"]="21"
data.loc[data["Age"]=="7          31", "Age"]="31"
data.loc[data["Age"]=="32  30", "Age"]="31"
data.loc[data["Age"]=="16 o 18", "Age"]="17"
data.loc[data["Age"]=="Edery", "Age"]="70"
data.loc[data["Age"]=="mid20s", "Age"]="25"
data.loc[data["Age"]=="Ca 33", "Age"]="33"
data.loc[data["Age"]=="21 or 26", "Age"]="23"
data.loc[data["Age"]=="18 o 22", "Age"]="20"
data.loc[data["Age"]=="9  12", "Age"]="12"
data.loc[data["Age"]=="9 monhs", "Age"]="1"
data.loc[data["Age"]=="25 o 35", "Age"]="27"
data.loc[data["Age"]=="23  26", "Age"]="24"
data.loc[data["Age"]=="33  37", "Age"]="35"
data.loc[data["Age"]=="25 or 28", "Age"]="27"
data.loc[data["Age"]=="37 67 35 27    27", "Age"]="40"
data.loc[data["Age"]=="21 3424  35", "Age"]="35"
data.loc[data["Age"]=="30  32", "Age"]="31"
data.loc[data["Age"]=="50  30", "Age"]="40"
data.loc[data["Age"]=="17  35", "Age"]="27"
data.loc[data["Age"]=="X", "Age"]="0"
data.loc[data["Age"]=="middeae", "Age"]="40"
data.loc[data["Age"]=="13 or 18", "Age"]="15"
data.loc[data["Age"]=="34  19", "Age"]="25"
data.loc[data["Age"]=="33  26", "Age"]="28"
data.loc[data["Age"]=="2 o 3 monhs", "Age"]="1"
data.loc[data["Age"]=="MAKE LINE GREEN", "Age"]="0"
data.loc[data["Age"]=="youn", "Age"]="1"
data.loc[data["Age"]=="7 or 8", "Age"]="7"
data.loc[data["Age"]=="17  16", "Age"]="17"
data.loc[data["Age"]=="F", "Age"]="0"
data.loc[data["Age"]=="Boh 11", "Age"]="11"
data.loc[data["Age"]=="9 or 10", "Age"]="9"
data.loc[data["Age"]=="36  23", "Age"]="32"
data.loc[data["Age"]=="AM", "Age"]="0"
data.loc[data["Age"]=="10 or 12", "Age"]="11"
data.loc[data["Age"]=="31 or 33", "Age"]="32"
data.loc[data["Age"]=="13 or 14", "Age"]="13"
data.loc[data["Age"]=="Teens", "Age"]="15"
data.loc[data["Age"]=="", "Age"]="0"
data.loc[data["Age"]=="adu", "Age"]="25"
data.loc[data["Age"]=="20s", "Age"]="25"
#ULTIMA VEZ, la proxima si valor no numerico se considera nulo y ya

In [78]:
data["Age"] = pd.to_numeric(data["Age"], errors='coerce').astype(int) #pasamos de STR a Int para separar por grupos de edad

In [79]:
get_unique_dtypes(data, "Age")

array([<class 'int'>], dtype=object)

In [80]:
data[~data['Age'].astype(str).str.isdigit()]

,Case_Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal_(Y/N),Time,Species,Investigator_Or_Source,Pdf,Href_Formula,Href,Case_Number_1,Case_Number_2,Original_Order,Time_groups


In [81]:
data["Age"] = data["Age"].apply(age_groups, age="Age")

TypeError: '<=' not supported between instances of 'str' and 'int'